# Optimizers in SciPy
This notebook is a very brief introduction to SciPy optimizers, documenting the example appendix/scipy_optim.py.

There are several optimizers in SciPy, in the module scipy.optimize. You can simply install them with +pip install scipy. 
You may find the user manual of this module in https://docs.scipy.org/doc/scipy/tutorial/optimize.html#tutorial-sqlsp.

In this serie of notebooks about robotics, we mostly use BFGS, a quasi-Newton constraint-free algorithm, and SLSQP, a sequential QP solver accepting both equality and inequality constraints.

We will then need the two +fmin functions from the scipy.optimize module, as well as +numpy to represent algebraic vectors.

In [ ]:
# %load appendix/generated/scipy_optim_import
import numpy as np
from scipy.optimize import fmin_bfgs, fmin_slsqp



They are generally following a similar API, taking as main argument the cost function to optimize +f, the initial guess +x0, and optiminally a callback function +callback and some constraints.

The cost objective should be defined as a function mapping the parameter space $x$ to a real value $f(x)$. Here is a simple polynomial example for $x \in R^2$:

In [ ]:
# %load appendix/generated/scipy_optim_cost
def cost(x):
    '''Cost f(x,y) = x^2 + 2y^2 - 2xy - 2x '''
    x0 = x[0]
    x1 = x[1]
    return -1 * (2 * x0 * x1 + 2 * x0 - x0**2 - 2 * x1**2)


The callback takes the same signature but returns nothing: it only works by side effect, for example printing something, or displaying some informations in a viewer or on a plot, or possibly storing data in a logger. Here is for example a callback written as the functor of an object, that can be used to adjust its behavior or store some data.

In [ ]:
# %load appendix/generated/scipy_optim_callback
class CallbackLogger:
    def __init__(self):
        self.nfeval = 1

    def __call__(self, x):
        print('===CBK=== {0:4d}   {1: 3.6f}   {2: 3.6f}'.format(self.nfeval, x[0], x[1], cost(x)))
        self.nfeval += 1


For BFGS, that's all we need, as it does not accept any additional constraints. 

In [ ]:
# %load appendix/generated/scipy_optim_bfgs
x0 = np.array([0.0, 0.0])
# Optimize cost without any constraints in BFGS, with traces.
xopt_bfgs = fmin_bfgs(cost, x0, callback=CallbackLogger())
print('\n *** Xopt in BFGS = %s \n\n\n\n' % str(xopt_bfgs))


In that case, the gradients of the cost are computed by BFGS using finite differencing (i.e. not very accurately, but the algorithmic cost is typically very bad). If you can provide some derivatives by yourself, it would greatly improve the result. Yet, as a first draft, it is generally not too bad.

For SLSQP, you can simply do the same.

In [ ]:
# %load appendix/generated/scipy_optim_without
# Optimize cost without any constraints in CLSQ
xopt_lsq = fmin_slsqp(cost, [-1.0, 1.0], iprint=2, full_output=1)
print('\n *** Xopt in LSQ = %s \n\n\n\n' % str(xopt_lsq))


Now, SLSQP can also handle explicit constraints. Equality and inequality constraints must be given separately as function from the parameter $x$ to a vector stacking all the numerical quantities, that must be null for equalities, and positive for inequalities.

We introduce here, as an example, two set of polynomial constraints.

In [ ]:
# %load appendix/generated/scipy_optim_constraints
def constraint_eq(x):
    ''' Constraint x^3 = y '''
    return np.array([x[0]**3 - x[1]])

def constraint_ineq(x):
    '''Constraint x>=2, y>=2'''
    return np.array([x[0] - 2, x[1] - 2])


The solver then run as follows:

In [ ]:
# %load appendix/generated/scipy_optim_with
# Optimize cost with equality and inequality constraints in CLSQ
xopt_clsq = fmin_slsqp(cost, [-1.0, 1.0], f_eqcons=constraint_eq, f_ieqcons=constraint_ineq, iprint=2, full_output=1)
print('\n *** Xopt in c-lsq = %s \n\n\n\n' % str(xopt_clsq))


That's all for now, folks.